In [2]:
import sklearn
import mlflow
import time
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.ensemble import RandomForestRegressor

In [3]:
mlflow_uri = "http://mlflow-server:8888"
mlflow.set_tracking_uri(mlflow_uri)
time.sleep(5)
#mlflow.set_experiment(mlflow_uri)
mlflow.set_experiment("v4_mlflow_onspark")

2025/05/04 03:10:43 INFO mlflow.tracking.fluent: Experiment with name 'v4_mlflow_onspark' does not exist. Creating a new experiment.


<Experiment: artifact_location='mlflow-artifacts:/893356954086239247', creation_time=1746328243340, experiment_id='893356954086239247', last_update_time=1746328243340, lifecycle_stage='active', name='v4_mlflow_onspark', tags={}>

In [4]:
import logging
logging.getLogger("mlflow").setLevel(logging.FATAL)

In [5]:
diamonds_df = sns.load_dataset('diamonds').drop(['cut', 'color', 'clarity'], axis=1)
#
X_train, X_test, y_train, y_test = train_test_split(diamonds_df.drop(["price"], axis=1), diamonds_df["price"], random_state=42)

X_train.head()

,carat,depth,table,x,y,z
35965,0.25,64.9,58.0,3.95,3.97,2.57
52281,0.84,61.8,56.0,6.04,6.07,3.74
6957,1.05,61.1,58.0,6.56,6.51,3.99
9163,1.02,60.7,56.0,6.53,6.50,3.95
50598,0.61,61.8,57.0,5.43,5.47,3.37


In [6]:
# Mlflow autolog
mlflow.sklearn.autolog()

# for GPU info please install this
# pip install pynvml

mlflow.enable_system_metrics_logging()
time.sleep(5)
with mlflow.start_run() as run :
    n_estimators = 110
    max_depth = 10
    max_features = 3

    rf =  RandomForestRegressor(n_estimators=n_estimators,max_depth=max_depth,max_features=max_features)
    rf.fit(X_train,y_train)

🏃 View run suave-duck-18 at: http://mlflow-server:8888/#/experiments/893356954086239247/runs/faddba62b9ae4308bd96f06d0196eab6
🧪 View experiment at: http://mlflow-server:8888/#/experiments/893356954086239247


In [7]:
model_uri = f"runs:/{run.info.run_id}/model"
print("model_uri : ",model_uri)
sk_model = mlflow.sklearn.load_model(model_uri)

/usr/local/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


model_uri :  runs:/faddba62b9ae4308bd96f06d0196eab6/model


In [8]:
sk_model.predict(X_test)

array([ 500.89074662, 1743.35826917, 1019.95436287, ..., 6975.08923591,
       5677.14827737, 5463.9340182 ])